In [1]:
import requests
import pandas as pd
from dateutil import parser


df = pd.read_csv('df_All_Coordinates.csv')

def parse_coordinates(coord):
    try:
        coord = eval(coord)  # Convert string to tuple
        if isinstance(coord, (list, tuple)) and len(coord) == 2:
            return coord[0], coord[1]
    except:
        return None, None

In [2]:
df["latitude"], df["longitude"] = zip(*df["tweet_coord"].apply(parse_coordinates))


df["latitude"] = df["latitude"].round(2)
df["longitude"] = df["longitude"].round(2)


df["tweet_created"] = df["tweet_created"].apply(lambda x: parser.parse(x[:-6]))  # Remove timezone offset
df["date"] = df["tweet_created"].dt.strftime("%Y-%m-%d")


weather_cache = {}

In [3]:

def get_weather_open_meteo(lat, lon, date):
    """Fetches historical weather for a given location & date using Open-Meteo API"""

    if pd.isna(lat) or pd.isna(lon):
        return None, None, None, None, None

    cache_key = (lat, lon, date) 

    if cache_key in weather_cache:
        return weather_cache[cache_key]  

    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&hourly=temperature_2m,relativehumidity_2m,precipitation,windspeed_10m,snowfall"

    try:
        response = requests.get(url).json()
        if "hourly" in response:
            # Extract the first hour's data (since we're querying for a single day)
            temp = response["hourly"]["temperature_2m"][0]
            humidity = response["hourly"]["relativehumidity_2m"][0]
            precipitation = response["hourly"]["precipitation"][0]
            wind_speed = response["hourly"]["windspeed_10m"][0]
            snowfall = response["hourly"]["snowfall"][0]

            # Store in cache
            weather_cache[cache_key] = (temp, humidity, precipitation, wind_speed, snowfall)
            return temp, humidity, precipitation, wind_speed, snowfall
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon} on {date}: {e}")
        return None, None, None, None, None

    return None, None, None, None, None

In [4]:

weather_data = df.apply(
    lambda row: get_weather_open_meteo(row["latitude"], row["longitude"], row["date"]), axis=1
)

In [5]:
weather_df = pd.DataFrame(weather_data.tolist(), columns=["temperature", "humidity", "precipitation", "wind_speed", "snowfall"])


df = df.join(weather_df)  


In [6]:
df.to_csv('example.csv', index=False)

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Encode sentiment labels
le = LabelEncoder()
df['sentiment_encoded'] = le.fit_transform(df['airline_sentiment'])

# Create a combined feature set with weather data and BERT embeddings
X_weather = df[['snowfall', 'wind_speed', 'precipitation', 'humidity', 'temperature']].values
X_bert = np.array(df['bert_embedding'].tolist())  # Extract BERT embeddings as an array
X_combined = np.hstack((X_weather, X_bert))  # Combine weather features with BERT embeddings

y = df['sentiment_encoded'].values  # Target sentiment variable


KeyError: 'bert_embedding'